In [1]:
from eda_import import *

In [2]:
print(datetime.datetime.today().date())

2018-05-08


In [3]:
%time df = pd.read_pickle('./results/all_cba.KLEAT.pkl')

CPU times: user 54.4 s, sys: 2min 25s, total: 3min 19s
Wall time: 3min 46s


In [4]:
TOTAL_NUM_CALLS = 67544140
assert df.shape[0] == TOTAL_NUM_CALLS

# Filter by target genes

In [5]:
TARGET_GENES = []
with open('./reference_data/target_genes.txt') as inf:
    for line in inf:
        TARGET_GENES.append(line.strip())

In [6]:
%time ndf = df[df.gene.isin(TARGET_GENES)]

CPU times: user 2min 27s, sys: 1min 45s, total: 4min 13s
Wall time: 4min 18s


In [7]:
ndf.shape

(11489199, 23)

In [8]:
ndf.head(1)

,gene,transcript,transcript_strand,coding,contig,chromosome,cleavage_site,within_UTR,distance_from_annotated_site,ESTs,length_of_tail_in_contig,number_of_tail_reads,number_of_bridge_reads,max_bridge_read_tail_length,bridge_read_identities,tail+bridge_reads,number_of_link_pairs,max_link_pair_length,link_pair_identities,hexamer_loc+id,3UTR_start_end,flag,kleat_fname
8,RTCA,ENST00000370126,+,yes,A0.S130911,chr1,100732395,yes,90,-,0,0,0,0,-,0,0,0,-,-,-,2,/projects/btl/zxue/tasrkleat-TCGA-results/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/kleat/cba.KLEAT


# Map kleat_fname to analysis_id

In [9]:
df_meta = pd.read_csv('./metadata/GCS_listing_27apr2016.csv.gz', compression='gzip',
                      usecols=['CGHubAnalysisID', 'GCSobject'])

df_meta = df_meta.rename(columns={'CGHubAnalysisID': 'analysis_id'})
df_meta = df_meta.drop_duplicates()

df_meta['kleat_fname'] = df_meta.GCSobject.apply(
    lambda x: x.replace('gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/', 
                        '/projects/btl/zxue/tasrkleat-TCGA-results/') + '/tasrkleat-results/kleat/cba.KLEAT')
df_meta.head(1)

,analysis_id,GCSobject,kleat_fname
0,00007994-abeb-4b16-a6ad-7230300a29e9,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga/COAD/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_1620885.c18465ae-447d-46c8-8b54-0156ab502265.sorted_genome_alignments.bam,/projects/btl/zxue/tasrkleat-TCGA-results/tcga/COAD/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_1620885.c18465ae-447d-46c8-8b54-0156ab502265.sorted_genome_alignments.bam/tasrkleat-results/kleat/cba.KLEAT


In [10]:
%time merged = ndf.merge(df_meta, how='left', on='kleat_fname')

CPU times: user 10.7 s, sys: 4.84 s, total: 15.5 s
Wall time: 15.5 s


In [11]:
merged.head(1)

,gene,transcript,transcript_strand,coding,contig,chromosome,cleavage_site,within_UTR,distance_from_annotated_site,ESTs,length_of_tail_in_contig,number_of_tail_reads,number_of_bridge_reads,max_bridge_read_tail_length,bridge_read_identities,tail+bridge_reads,number_of_link_pairs,max_link_pair_length,link_pair_identities,hexamer_loc+id,3UTR_start_end,flag,kleat_fname,analysis_id,GCSobject
0,RTCA,ENST00000370126,+,yes,A0.S130911,chr1,100732395,yes,90,-,0,0,0,0,-,0,0,0,-,-,-,2,/projects/btl/zxue/tasrkleat-TCGA-results/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/kleat/cba.KLEAT,1d811d54-caa1-41f3-bfc2-400eebdd97ef,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz


# Remove unused columns

In [12]:
wanted_cols = [
    'gene',
    'transcript_strand',
    'chromosome',
    'cleavage_site',
    'length_of_tail_in_contig',
    'number_of_bridge_reads',
    'max_bridge_read_tail_length',
    'analysis_id'
]

%time odf = merged[wanted_cols].copy()

CPU times: user 6.5 s, sys: 13.1 s, total: 19.6 s
Wall time: 19.6 s


In [13]:
odf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11489199 entries, 0 to 11489198
Data columns (total 8 columns):
gene                           object
transcript_strand              object
chromosome                     object
cleavage_site                  int64
length_of_tail_in_contig       int64
number_of_bridge_reads         int64
max_bridge_read_tail_length    int64
analysis_id                    object
dtypes: int64(4), object(4)
memory usage: 788.9+ MB


In [15]:
odf.rename(columns={
    'gene': 'gene_name',
    'cleavage_site': 'clv', 
    'transcript': 'transcript_id',
    'transcript_strand': 'strand',
    'chromosome': 'seqname'}, inplace=True)

In [16]:
ON_TARGET_NUM_CALLS = 11489199
assert odf.shape[0] == ON_TARGET_NUM_CALLS

In [17]:
odf.head(1)

,gene_name,strand,seqname,clv,length_of_tail_in_contig,number_of_bridge_reads,max_bridge_read_tail_length,analysis_id
0,RTCA,+,chr1,100732395,0,0,0,1d811d54-caa1-41f3-bfc2-400eebdd97ef


In [18]:
assert odf.gene_name.unique().shape[0] == 114

In [20]:
print('{0:.3%} data remained'.format(odf.shape[0] / TOTAL_NUM_CALLS))

17.010% data remained


In [21]:
%time odf.to_pickle('./results/all_cba.KLEAT.on-target-cleaned.pkl')

CPU times: user 3.02 s, sys: 5.68 s, total: 8.7 s
Wall time: 11.6 s
